In [54]:
import pymongo
import pandas as pd
import numpy as np

In [55]:
cliente=pymongo.MongoClient('mongodb://localhost:27017')  # cliente , la conexion por defecto
db=cliente.EmptiedSpain   # llamada a la base de datos


In [56]:
hospitales=db.emptiedSpain.find({'Entidad': 'Hospital'})

In [57]:
df = pd.DataFrame(hospitales)

In [58]:
def cleanNans(dic):
    a = dic['coordinates']
    if np.nan in a: 
        dic['coordinates'] = [0,0]
    return dic

In [59]:
df.location = df.location.apply(cleanNans)

In [60]:
df.head()

,_id,NOMBRE,DIRECCION,TELEFONO,CODMU,MUNICIPIOS,CODPROV,PROVINCIAS,COMUNIDADES,CODPOSTAL,NCAMAS,FINALIDAD_ASISITENCIAL,DEPENDENCIA_PATRIMONIAL,location,Entidad
0,50013.0,HOSPITAL NUESTRA SEÑORA DE SONSOLES,"AV. JUAN CARLOS I, S/N",920358000.0,50198.0,Ávila,5.0,ÁVILA,CASTILLA Y LEÓN,5071.0,0.0,GENERAL,SEGURIDAD SOCIAL,"{'type': 'Point', 'coordinates': [-4.666844434...",Hospital
1,50034.0,HOSPITAL PROVINCIAL DE ÁVILA,"JESÚS DEL GRAN PODER, 42",920357200.0,50198.0,Ávila,5.0,ÁVILA,CASTILLA Y LEÓN,5003.0,0.0,GENERAL,COMUNIDAD AUTÓNOMA,"{'type': 'Point', 'coordinates': [-4.692580594...",Hospital
2,50049.0,CLÍNICA SANTA TERESA,"AV. SANTA CRUZ DE TENERIFE, 11",920252525.0,50198.0,Ávila,5.0,ÁVILA,CASTILLA Y LEÓN,5005.0,33.0,GENERAL,PRIVADO NO BENÉFICO,"{'type': 'Point', 'coordinates': [-4.686130078...",Hospital
3,50061.0,COMPLEJO ASISTENCIAL DE AVILA,"AVDA. JUAN CARLOS I, S/N",920358000.0,50198.0,Ávila,5.0,ÁVILA,CASTILLA Y LEÓN,5071.0,430.0,GENERAL,ENTIDADES PÚBLICAS,"{'type': 'Point', 'coordinates': [-4.652362102...",Hospital
4,50072.0,HOSPITAL DE SALUD MENTAL CASTA SALUD ARÉVALO,"PASEO SAN JUAN BOSCO, 2",920301400.0,50163.0,Arévalo,5.0,ÁVILA,CASTILLA Y LEÓN,5200.0,80.0,PSIQUIÁTRICO,PRIVADO NO BENÉFICO,"{'type': 'Point', 'coordinates': [-4.716772079...",Hospital


In [61]:
db.emptiedSpain.create_index([('location', '2dsphere')])


'location_2dsphere'

In [95]:
def find_near(geopoint, radio=1000):
    return db.emptiedSpain.find({'$and':[{'location':{
        '$near': {'$geometry': geopoint, '$maxDistance':radio}
    }},{'Entidad': 'Hospital'}]})

In [96]:
pueblos=db.emptiedSpain.find({'Entidad': 'Entidad de población'})

In [97]:
df_p = pd.DataFrame(pueblos)

In [146]:
i = 1050
a = df_p.location[i]
df_p.iloc[i]

_id                                                                       9249000100
Comunidad Autónoma                                                   Castilla y León
Provincia                                                                     Burgos
Municipio                                                         Palazuelos de Muñó
Entidad Singular de Población                                     PALAZUELOS DE MUÑÓ
Cobertura ≥ 30Mbps                                                              0.74
Cobertura redes fijas ≥ 100Mbps                                                  0.0
Cobertura 3G HSPA                                                                1.0
Cobertura 4G LTE                                                                 1.0
Weather_info                       En Palazuelos de Muñó, los veranos son cortos,...
pob_tot_muni                                                                   530.0
partido_gov                                                      

In [147]:
a

{'type': 'Point', 'coordinates': [3.9883333333333, 42.196388888889]}

In [149]:
pd.DataFrame(find_near(a, radio=600000))

,_id,NOMBRE,DIRECCION,TELEFONO,CODMU,MUNICIPIOS,CODPROV,PROVINCIAS,COMUNIDADES,CODPOSTAL,NCAMAS,FINALIDAD_ASISITENCIAL,DEPENDENCIA_PATRIMONIAL,location,Entidad
0,420026.0,HOSPITAL VIRGEN DEL MIRÓN,"CTRA. DE LOGROÑO, 8",975220850.0,421736.0,Soria,42.0,SORIA,CASTILLA Y LEÓN,42005.0,0.0,GERIATRÍA Y/O LARGA ESTANCIA,COMUNIDAD AUTÓNOMA,"{'type': 'Point', 'coordinates': [-2.463960482...",Hospital
1,420011.0,HOSPITAL SANTA BÁRBARA,"Pº SANTA BÁRBARA, S/N",975234300.0,421736.0,Soria,42.0,SORIA,CASTILLA Y LEÓN,42005.0,0.0,GENERAL,SEGURIDAD SOCIAL,"{'type': 'Point', 'coordinates': [-2.471273765...",Hospital
2,420040.0,COMPLEJO ASISTENCIAL DE SORIA,"Pº. STA. BÁRBARA, S/N",975234300.0,421736.0,Soria,42.0,SORIA,CASTILLA Y LEÓN,42005.0,319.0,GENERAL,ENTIDADES PÚBLICAS,"{'type': 'Point', 'coordinates': [-2.472561225...",Hospital
3,90098.0,HOSPITAL SANTIAGO APÓSTOL,"CTRA. DE ORÓN, S/N",947349000.0,92194.0,Miranda de Ebro,9.0,BURGOS,CASTILLA Y LEÓN,9200.0,114.0,GENERAL,SEGURIDAD SOCIAL,"{'type': 'Point', 'coordinates': [-2.989475378...",Hospital
